<h1> Preprocessing </h1>

In [1]:
import pandas as pd
df = pd.read_csv('survey lung cancer.csv')
df.head()

GENDER AGE SMOKING  YELLOW_FINGERS  ANXIETY PEER_PRESSURE CHRONIC DISEASE  \
0      M  69       1               2        2             1               1   
1      M  74       2               1        1             1               2   
2      F  59       1               1        1             2               1   
3      M  63       2               2        2             1               1   
4      F  63       1               2        1             1               1   

  FATIGUE  ALLERGY  WHEEZING ALCOHOL CONSUMING COUGHING SHORTNESS OF BREATH  \
0        2        1        2                 2        2                   2   
1        2        2        1                 1        1                   2   
2        2        1        2                 1        2                   2   
3        1        1        1                 2        1                   1   
4        1        1        2                 1        2                   2   

  SWALLOWING DIFFICULTY  CHEST PAIN LUNG_CANCER  
0                     2           2         YES  
1                     2           2         YES  
2                     1           2          NO  
3                     2           2          NO  
4                     1           1          NO

In [102]:
df.isnull().sum()

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL CONSUMING        0
COUGHING                 0
SHORTNESS OF BREATH      0
SWALLOWING DIFFICULTY    0
CHEST PAIN               0
LUNG_CANCER              0
dtype: int64

In [2]:
import numpy as np
df['LUNG_CANCER'] = df['LUNG_CANCER'].map({'YES': 1, 'NO': 0})
df['GENDER'] = df['GENDER'].map({'F': 1, 'M': 0})

df.replace(['X', 'x'], np.nan, inplace=True)

for column in df.columns:
    if df[column].dtype == object and not pd.isnull(df[column]).all():
        df[column] = pd.to_numeric(df[column],downcast="integer", errors='coerce')
    if column not in ['GENDER', 'LUNG_CANCER', 'AGE']:
        df[column] = df[column].apply(lambda x: x-1 if pd.notnull(x) and x in [1, 2] else x)
df.head()

GENDER   AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0       0  69.0      0.0               1        1            0.0   
1       0  74.0      1.0               0        0            0.0   
2       1  59.0      0.0               0        0            1.0   
3       0  63.0      1.0               1        1            0.0   
4       1  63.0      0.0               1        0            0.0   

   CHRONIC DISEASE  FATIGUE   ALLERGY   WHEEZING  ALCOHOL CONSUMING  COUGHING  \
0              0.0       1.0       0.0       1.0                1.0       1.0   
1              1.0       1.0       1.0       0.0                0.0       0.0   
2              0.0       1.0       0.0       1.0                0.0       1.0   
3              0.0       0.0       0.0       0.0                1.0       0.0   
4              0.0       0.0       0.0       1.0                0.0       1.0   

   SHORTNESS OF BREATH  SWALLOWING DIFFICULTY  CHEST PAIN  LUNG_CANCER  
0                  1.0                    1.0           1            1  
1                  1.0                    1.0           1            1  
2                  1.0                    0.0           1            0  
3                  0.0                    1.0           1            0  
4                  1.0                    0.0           0            0

<h2> Implementation Without Library </h2>

In [3]:
from sklearn.model_selection import train_test_split
X = df.drop('LUNG_CANCER', axis=1)
y = df['LUNG_CANCER']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def class_probabilities(y_train):
    yes_count = (y_train == 1).sum()
    no_count = (y_train == 0).sum()
    total_count = len(y_train)
    yes_probability = yes_count / total_count
    no_probability = no_count / total_count
    return yes_probability, no_probability


def predict(X_test, yes_probability, no_probability):
    predictions = []
    for index, row in X_test.iterrows():
        yes_prob_product = yes_probability
        no_prob_product = no_probability
        for feature in X_test.columns:
            feature_value = row[feature]
            if feature_value > 0.5:
                yes_prob_product *= feature_value
                no_prob_product *= (1 - feature_value)
            else:
                yes_prob_product *= (1 - feature_value)
                no_prob_product *= feature_value
        prediction = 1 if yes_prob_product > no_prob_product else 0
        predictions.append(prediction)
    return predictions



yes_probability, no_probability = class_probabilities(y_train)

predictions = predict(X_test, yes_probability, no_probability)
correct_predictions = (y_test == predictions).sum()
total_predictions = len(y_test)
accuracy = correct_predictions / total_predictions

accuracy


0.9193548387096774

In [5]:
true_positives = false_positives = true_negatives = false_negatives = 0

for actual, predicted in zip(y_test, predictions):
    if actual == 1 and predicted == 1:
        true_positives += 1
    elif actual == 0 and predicted == 1:
        false_positives += 1
    elif actual == 1 and predicted == 0:
        false_negatives += 1
    elif actual == 0 and predicted == 0:
        true_negatives += 1

precision = true_positives / (true_positives + false_positives) if true_positives + false_positives else 0
recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives else 0
confusion_matrix = [[true_positives, false_positives], [false_negatives, true_negatives]]

precision, recall, confusion_matrix

(0.9661016949152542, 0.95, [[57, 2], [3, 0]])

<h2> Implementation with SKLEARN </h2>

In [9]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

gnb = GaussianNB()
gnb.fit(X_train_imputed, y_train)

gnb_predictions = gnb.predict(X_test_imputed)

gnb_accuracy = accuracy_score(y_test, gnb_predictions)
gnb_precision = precision_score(y_test, gnb_predictions)
gnb_recall = recall_score(y_test, gnb_predictions)
gnb_conf_matrix = confusion_matrix(y_test, gnb_predictions)

gnb_accuracy, gnb_precision, gnb_recall, gnb_conf_matrix


(0.967741935483871,
 0.9833333333333333,
 0.9833333333333333,
 array([[ 1,  1],
        [ 1, 59]], dtype=int64))

<h2>Prediction of Dataset 2</h2>

In [14]:
phishing_df = pd.read_csv('web-page-phishing.csv')
for column in df.columns:
    if df[column].dtype == object and not pd.isnull(df[column]).all():
        df[column] = pd.to_numeric(df[column],downcast="integer", errors='coerce')
phishing_df.head()

url_length  n_dots  n_hypens  n_underline  n_slash  n_questionmark  \
0          37       3         0            0        0               0   
1          77       1         0            0        0               0   
2         126       4         1            2        0               1   
3          18       2         0            0        0               0   
4          55       2         2            0        0               0   

   n_equal  n_at  n_and  n_exclamation  n_space  n_tilde  n_comma  n_plus  \
0        0     0      0              0        0        0        0       0   
1        0     0      0              0        0        0        0       0   
2        3     0      2              0        0        0        0       0   
3        0     0      0              0        0        0        0       0   
4        0     0      0              0        0        0        0       0   

   n_asterisk  n_hastag  n_dollar  n_percent  n_redirection  phishing  
0           0         0         0          0              0         0  
1           0         0         0          0              1         1  
2           0         0         0          0              1         1  
3           0         0         0          0              1         0  
4           0         0         0          0              1         0

In [31]:
from sklearn.preprocessing import StandardScaler
X_phishing = phishing_df.drop('phishing', axis=1)
y_phishing = phishing_df['phishing']

X_phishing_train, X_phishing_test, y_phishing_train, y_phishing_test = train_test_split(X_phishing, y_phishing, test_size=0.2, random_state=42)



yes_probability, no_probability = class_probabilities(y_phishing_train)

predictions = predict(X_phishing_test, yes_probability, no_probability)
correct_predictions = (y_phishing_test == predictions).sum()
total_predictions = len(y_phishing_test)
accuracy = correct_predictions / total_predictions
accuracy

0.365607513988809

In [21]:
true_positives = false_positives = true_negatives = false_negatives = 0

for actual, predicted in zip(y_phishing_test, predictions):
    if actual == 1 and predicted == 1:
        true_positives += 1
    elif actual == 0 and predicted == 1:
        false_positives += 1
    elif actual == 1 and predicted == 0:
        false_negatives += 1
    elif actual == 0 and predicted == 0:
        true_negatives += 1

precision = true_positives / (true_positives + false_positives) if true_positives + false_positives else 0
recall = true_positives / (true_positives + false_negatives) if true_positives + false_negatives else 0
confusion_matrix = [[true_positives, false_positives], [false_negatives, true_negatives]]

precision, recall, confusion_matrix

(0.365607513988809, 1.0, [[7318, 12698], [0, 0]])

<h2> Second Implementation Without Library </h2>
This one is with the help of others Code

In [28]:
def calculate_likelihoods(X_train, y_train):
    classes = np.unique(y_train)
    features = X_train.shape[1]
    likelihoods = {}
    
    for cls in classes:
        likelihoods[cls] = {}
        for feature_index in range(features):
            feature_values = np.unique(X_train.iloc[:, feature_index])
            likelihoods[cls][feature_index] = {}
            for value in feature_values:
                feature_class_df = X_train[y_train == cls]
                likelihood = len(feature_class_df[feature_class_df.iloc[:, feature_index] == value]) / len(feature_class_df)
                likelihoods[cls][feature_index][value] = likelihood if likelihood > 0 else 1e-10
                
    return likelihoods

def naive_bayes_predict_optimized(X_test, likelihoods, prior):
    predictions = []
    classes = list(prior.keys())
    
    for i in range(len(X_test)):
        row = X_test.iloc[i, :]
        class_probabilities = {}
        
        for cls in classes:
            class_probability = prior[cls]
            for j in range(len(row)):
                value = row.iloc[j]
                if value in likelihoods[cls][j]:
                    class_probability *= likelihoods[cls][j][value]
                else:
                    class_probability *= 1e-10
            class_probabilities[cls] = class_probability
            
        predicted_class = max(class_probabilities, key=class_probabilities.get)
        predictions.append(predicted_class)
        
    return predictions

def calculate_prior(y_train):
    """Calculate the prior probabilities of each class."""
    classes = np.unique(y_train)
    prior = {cls: len(y_train[y_train == cls]) / len(y_train) for cls in classes}
    return prior


X_phishing_train, X_phishing_test, y_phishing_train, y_phishing_test = train_test_split(X_phishing, y_phishing, test_size=0.2, random_state=42)

prior = calculate_prior(y_train)
likelihoods = calculate_likelihoods(X_phishing_train, y_phishing_train)

predictions_optimized = naive_bayes_predict_optimized(X_phishing_test, likelihoods, prior)

y_true = y_phishing_test.values
y_pred = predictions_optimized

tp = fp = tn = fn = 0
for i in range(len(y_true)):
    if y_true[i] == y_pred[i] == 1:
        tp += 1
    if y_pred[i] == 1 and y_true[i] != y_pred[i]:
        fp += 1
    if y_true[i] == y_pred[i] == 0:
        tn += 1
    if y_pred[i] == 0 and y_true[i] != y_pred[i]:
        fn += 1
        
accuracy = (tp + tn) / len(y_true)
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0
confusion_matrix = np.array([[tp, fp], [fn, tn]])


accuracy, precision, recall, confusion_matrix

(0.8509692246203038,
 0.7419895054147594,
 0.9081716315933315,
 array([[ 6646,  2311],
        [  672, 10387]]))

<h2>DATASET 2 with the help of SKLEARN </h2>

In [30]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

gnb = GaussianNB()
gnb.fit(X_phishing_train, y_phishing_train)

predictions_sklearn = gnb.predict(X_phishing_test)

accuracy_sklearn = accuracy_score(y_phishing_test, predictions_sklearn)
precision_sklearn = precision_score(y_phishing_test, predictions_sklearn)
recall_sklearn = recall_score(y_phishing_test, predictions_sklearn)
confusion_matrix_sklearn = confusion_matrix(y_phishing_test, predictions_sklearn)

accuracy_sklearn, precision_sklearn, recall_sklearn, confusion_matrix_sklearn

(0.7156274980015987,
 0.8864068441064639,
 0.25485105220005466,
 array([[12459,   239],
        [ 5453,  1865]], dtype=int64))